# Spike Challenge Octubre 2019
## Ivan Jara Varela

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline

Importadas las librerías a utilizar, comenzamos:

## Número 1

Descarga del archivo csv para crear el dataframe.

In [ ]:
#df = pd.read_gbq('SELECT * FROM public.caudal_extra_min',project_id='spikelab') 
## Para base de datos en BigQuery (no tenía acceso)

df = pd.read_csv('caudal_extra.csv')

## Número 2

Hay varias filas con datos faltantes, porque no todas las cuencas tienen estaciones de medición de precipitación o temperatura.

In [ ]:
num_rows_nan_t = df['temp_max_promedio'].isna().sum()
num_rows_nan_p = df['precip_promedio'].isna().sum()
print('Hay ',num_rows_nan_t,' filas sin registro de temperatura')
print('Hay ',num_rows_nan_p,' filas sin registro de precipitación')

Podemos ver un resumen de las tres principales variables

In [ ]:
print(df[['caudal','temp_max_promedio','precip_promedio']].describe())

También podemos ver si la distribución de las variables es asimétrica

In [ ]:
print(df[['caudal','temp_max_promedio','precip_promedio']].skew())

Las mediciones de caudal y precipitación se concentran más a la derecha (mediciones de caudal mayores al promedio) y las de temperatura a la izquierda (menores al promedio), considerando una distribución normal.

Hay dos columnas iguales, gauge_id y codigo_estacion, por lo que se elimina una

In [ ]:
print(set(df['gauge_id'] == df['codigo_estacion']))
df.drop(columns='gauge_id',inplace=True)

Hay columnas inútiles, como institución y fuente, porque tienen el mismo valor en todas las filas (mediciones)

In [ ]:
print('Hay ',len(set(df['institucion'])),' valor para todas las filas de la columna institución')
print('Hay ',len(set(df['fuente'])),' valor para todas las filas de la columna fuente')
df.drop(columns=['institucion','fuente'],inplace=True)

También se puede saber cuantas cuencas y estaciones hay

In [ ]:
num_cuencas = len(set(df['codigo_cuenca']))
num_estaciones = len(set(df['codigo_estacion']))
print('Hay ',num_cuencas,' cuencas, monitoreadas en ',num_estaciones,' estaciones')

Finalmente, dejamos solo las variables numéricas, transformamos la columna fecha a formato datetime y eliminamos el dataframe original

In [ ]:
dff = df[['codigo_estacion',
          'codigo_cuenca',
          'cantidad_observaciones',
          'altura',
          'latitud',
          'longitud',
          'fecha',
          'caudal',
          'precip_promedio',
          'temp_max_promedio']]
dff = df
dff['fecha'] = pd.to_datetime(dff['fecha'])

del df

## Número 3

### a)

Las estaciones, mientras existen, tienen registros de caudal en todo el periodo; para la temperatura y la precipitación no es el caso.

In [ ]:
def time_plot_una_estacion(codigo_estacion,columna,fecha_min,fecha_max):
    df1 = dff[dff['codigo_estacion'] == codigo_estacion]
    df1.set_index('fecha',inplace=True)
    df1 = df1[[columna]]
    df1 = df1[fecha_min:fecha_max]
    return df1.plot(title = 'Código estación: ' + str(codigo_estacion))

A continuación hay un ejemplo de la implementación de la función.

In [ ]:
time_plot_una_estacion(4540001,'caudal','1965-01-06','1976-02-13')

### b)

In [ ]:
def time_plot_estaciones_varias_columnas(codigo_estacion,columnas,fecha_min,fecha_max):
    df1 = dff[dff['codigo_estacion'] == codigo_estacion]
    df1.set_index('fecha',inplace=True)
    df1 = df1[columnas]
    df1 = 100*(df1 - df1.min())/(df1.max()- df1.min())
    df1 = df1[fecha_min:fecha_max]
    return df1.plot(title= 'est: ' + str(codigo_estacion) + ', normalizado con min/max')

Un ejemplo de la implementación:

In [ ]:
time_plot_estaciones_varias_columnas(4540001,['caudal','precip_promedio','temp_max_promedio'],'1980-01-06','1985-02-13')

A modo de "prueba" se puede ver como la temperatura varía al pasar el año, lo que indica que parece estar bien.

# Número 4

Utilizando una muy buena forma de transformar el formato datetime a meteorological season (verano,invierno,primavera y otoño), adaptada desde [aquí](https://stackoverflow.com/questions/44124436/python-datetime-to-season)

In [ ]:
dff['season'] = [(dt.month%12 + 3)//3 for dt in dff['fecha']]

Se crea una función para calcular el percentil 95

In [ ]:
def per95(x):
    return np.percentile(x,95)

Se crea un nuevo dataframe con pivot_table, donde los índices sean codigo_estacion y season, y los valores el percentil 95 de las 3 variables  en estudio

In [ ]:
df_q = pd.pivot_table(dff,index=['codigo_estacion','season'],values=['caudal','precip_promedio','temp_max_promedio'],\
                      aggfunc=per95)

Se crea una función para asignar 1 si un número es mayor, y 0 si es menor a otro

In [ ]:
def mm(x,y):
    if (x >= y):
        return 1
    else:
        return 0

Ahora, se aplica la función anterior comparando el dataframe df_q y dff, creando las columnas requeridas, para luego eliminar df_q y los arrays temporales. Este toma 5 minutos en mi computador

In [ ]:
c_xtrem = []
p_xtrem = []
t_xtrem = []
for dc,dp,dt,de,ds in zip(dff['caudal'],dff['precip_promedio'],\
                          dff['temp_max_promedio'],\
                          dff['codigo_estacion'],dff['season']):
    xtrem = df_q.loc[(de,ds)]
    c_xtrem.append(mm(dc,xtrem[0]))
    p_xtrem.append(mm(dp,xtrem[1]))
    t_xtrem.append(mm(dt,xtrem[2]))


dff['caudal_extremo'] = c_xtrem
dff['precip_extremo'] = p_xtrem
dff['temp_extremo'] = t_xtrem
del df_q, c_xtrem, p_xtrem, t_xtrem

In [ ]:
dff[['codigo_estacion','season','caudal_extremo','precip_extremo','temp_extremo']].head()

con 1:verano 2:otoño 3:invierno y 4:primavera

El filtro por el percentil 95 es una buena manera de capturar los eventos extremos. Otra forma sería aprovechar la aleatoriedad de los datos; se me ocurre que la distribución de eventos extremos en el tiempo tiene más entropía, pero por ahora no sé como calcular algo así.

# Número 5

Primero, se crea un nuevo dataframe, y utilizamos nuevamente pivot_tablet para hacer el aggregate de eventos extremos de caudal por cuenca

In [ ]:
df_caudal = dff.pivot_table(index='codigo_cuenca',values='caudal_extremo',aggfunc=np.sum)

Luego, se grafica el número de ocurrencias de eventos extremos de caudal por cuenca

In [ ]:
df_caudal.plot.bar(title='N° de ocurrencias por cuenca')

Podemos ver como la cantidad de eventos extremos de caudal entre algunas cuencas difiere en 1 orden de magnitud (10 veces más grandes) en algunos casos.

# Número 6

Se crea una nueva columna, 'year', y un nuevo dataframe, df6

In [ ]:
years = [dt.year for dt in dff['fecha']]
dff['year'] = years
df6 = dff.drop(columns='fecha')

Por última vez, se utiliza pivot_table para efectuar el aggregate de las variables, esta vez con la siguiente función, que entrega el porcentaje de eventos extremos de cada variable en cada año

In [ ]:
def perc100(x):
    return 100*np.mean(x)

In [ ]:
df6 = pd.pivot_table(dff,index='year',values=['caudal_extremo','precip_extremo','temp_extremo'],aggfunc=perc100)

Finalmente, se grafica y se eliminan las variables temporales

In [ ]:
df6.plot(title = 'Eventos extremos por año') 
del years, df6

Solo observando el gráfico anterior, los eventos de precipitación extrema han disminuido, los de caudal no parecen mostrar una tendencia y los de temperatura parecen subir. Considerar por separado cuencas, zonas geográficas u otras funciones de aggregate quizás muestren mas claramente alguna tendencia.

# Número 7

Se crean las variables 'month' y 'day', y se eliminan las filas que contengan valores missing

In [ ]:
dff['month'] = [dt.month for dt in dff['fecha']]
dff['day'] = [dt.day for dt in dff['fecha']]
dff.dropna(inplace=True)

Se fija el tamaño del dataset de prueba en un 25%

In [ ]:
test_s = 0.25

Se divide el dataset en features(datax) y objetivo(datay)

In [ ]:
datax = dff[['codigo_estacion',
          'codigo_cuenca',
          'cantidad_observaciones',
          'altura',
          'latitud',
          'longitud',
          'year',
          'month',
          'day',
          'caudal',   
          'precip_promedio',
          'temp_max_promedio']]


datay = dff[['caudal_extremo']]

Se divide entre los train y test dataset

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(datax,datay,test_size=test_s)

Se entrenará un modelo K-Nearest Neighbors (KNN), con un radio de k=15, y ponderado por la distancia. El entrenamiento me tomó alrededor de 2 minutos

In [ ]:
knn = KNeighborsClassifier(n_neighbors=15,weights='distance')
knn.fit(x_train,y_train.values.ravel())
y_pred = knn.predict(x_test)

Intenté ocupar todos los datos disponibles originales. Propongo utilizar las 11 variables para predecir caudal extremo en el futuro, o sea, simplemente ingresar los 11 valores requeridos del día a pronosticar. Como lo entiendo, más que predecir condiciones futuras, el modelo aprende los patrones cíclicos y la tendencia creciente o decreciente de la variable caudal extremo. 

KNN funciona en base a distancias euclidianas, por lo que en principio no hay restricciones numéricas para las features. Debería funcionar bien con datos razonables y dentro de las variaciones convencionales (no utilizar 60°C en temperatura, por ejemplo).

# Número 8

## a)

Utilizando la métrica más sencilla, que es la comparación entre los datos predichos y el dataset de prueba apartado en el punto anterior, se tiene que:

In [ ]:
print('El modelo tiene una performance de ', round(metrics.accuracy_score(y_test, y_pred)*100,4),'%')

Para KNN no conozco una manera directa de hacer análisis de sensibilidad, pero hay variables que quizás esten entregando la misma información (redundantes), como la cuenca, estación y las coordenadas.

Quizás sirve "apagar" una de las features, y entrenar el modelo para ver como afecta en la metrica utilizada para medir performance.

A pesar de lo anterior, el entrenamiento no tarda mucho, y parece entregar buenos resultados. Es un buen modelo para comenzar.

## b)

No comprendo la pregunta; en los modelos que conozco (y este en particular) creo que no se puede elegir exactamente qué datos se utilizaran para entrenar el modelo. Si la pregunta consiste en ocupar el 70% del dataset para entrenar, entonces

In [ ]:
test_s = 0.3

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(datax,datay,test_size=test_s)

knn1 = KNeighborsClassifier(n_neighbors=15,weights='distance')
knn1.fit(x_train,y_train.values.ravel())
y_pred = knn1.predict(x_test)

In [ ]:
print('El modelo tiene una performance de ', round(metrics.accuracy_score(y_test, y_pred)*100,4),'%')

Me parece que sigue siendo útil, porque es muy similar al resultado anterior.